In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,48981
2,Huelva,Confirmados PDIA 14 días,1558
3,Huelva,Tasa PDIA 14 días,"303,6030944911043"
4,Huelva,Confirmados PDIA 7 días,992
5,Huelva,Tasa PDIA 7 dias,"193,30826042052342"
6,Huelva,Total Confirmados,49189
7,Huelva,Curados,46217
8,Huelva,Fallecidos,424


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  48981.0


/tmp/ipykernel_27479/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  13666.0


/tmp/ipykernel_27479/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_27479/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_27479/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_27479/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 439 personas en los últimos 7 días 

Un positivo PCR cada 274 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,48981.0,992.0,1558.0,513170.0,193.308260,303.603094,253.0
Huelva-Costa,28914.0,564.0,877.0,289548.0,194.786357,302.885877,158.0
Huelva (capital),13666.0,328.0,524.0,143837.0,228.035902,364.301258,84.0
Condado-Campiña,15244.0,264.0,461.0,156231.0,168.980548,295.075881,54.0
Sierra de Huelva-Andévalo Central,4392.0,161.0,217.0,67391.0,238.904305,322.001454,41.0
Aracena,749.0,67.0,72.0,8255.0,811.629316,872.198667,23.0
Cartaya,2315.0,33.0,66.0,20083.0,164.318080,328.636160,16.0
Lepe,3108.0,39.0,46.0,27880.0,139.885222,164.992826,14.0
Aljaraque,1772.0,57.0,83.0,21474.0,265.437273,386.513924,14.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Puerto Moral,14.0,9.0,9.0,278.0,3237.410072,3237.410072,0.0
Granada de Río-Tinto (La),18.0,1.0,4.0,243.0,411.522634,1646.090535,0.0
Cumbres Mayores,164.0,24.0,26.0,1749.0,1372.212693,1486.563751,7.0
Paterna del Campo,284.0,10.0,46.0,3457.0,289.268152,1330.633497,4.0
Aracena,749.0,67.0,72.0,8255.0,811.629316,872.198667,23.0
Villanueva de las Cruces,24.0,3.0,3.0,387.0,775.193798,775.193798,1.0
Corteconcepción,51.0,2.0,4.0,536.0,373.134328,746.268657,0.0
Bollullos Par del Condado,1434.0,43.0,80.0,14387.0,298.880934,556.057552,13.0
Punta Umbría,1370.0,43.0,79.0,15355.0,280.039075,514.490394,8.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Manzanilla,153.0,1.0,8.0,2118.0,47.214353,377.714825,0.0,0.125000
Paterna del Campo,284.0,10.0,46.0,3457.0,289.268152,1330.633497,4.0,0.217391
Higuera de la Sierra,75.0,1.0,4.0,1291.0,77.459334,309.837335,0.0,0.250000
Granada de Río-Tinto (La),18.0,1.0,4.0,243.0,411.522634,1646.090535,0.0,0.250000
Jabugo,89.0,1.0,3.0,2260.0,44.247788,132.743363,0.0,0.333333
Escacena del Campo,242.0,2.0,5.0,2287.0,87.450809,218.627022,0.0,0.400000
Zalamea la Real,131.0,2.0,5.0,3054.0,65.487885,163.719712,1.0,0.400000
Villalba del Alcor,505.0,1.0,2.0,3366.0,29.708853,59.417706,0.0,0.500000
Cartaya,2315.0,33.0,66.0,20083.0,164.318080,328.636160,16.0,0.500000
